- set up environment

In [102]:
# set up environment

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

# for Set working folder, etc.
import os
import os.path

# for sampling 
import random

# set working folder
# fw slashes for windows
cwd_desired = "C:/Users/Bonita/Documents/work/velexi/neural networks" 
os.chdir(cwd_desired) 

# test code
# print(os.getcwd())                           

- ETL spectrum files

In [103]:
# etl spectrum files

# turn files into lists and cut off first line
# put into database "spectra"
# turn deleted points (flagged as -1.23e+34) into 0's


# init spectra and class databases
spectra_wd = str(cwd_desired + "/spectra2")
num_samples = len([name for name in os.listdir(spectra_wd) if \
                   os.path.isfile(os.path.join(spectra_wd,name))])
# test code 
print(num_samples)

spectrum_len = 480                                 # 480 >= the BECK spectrometer spectrum length
spectra = np.zeros((num_samples,spectrum_len))             
y = np.zeros((num_samples, 1))

# fill databases
i = 0

# find spectra folder
os.chdir(spectra_wd)

# etl the data
for filename in os.listdir(spectra_wd):
    file_object  = open(filename, 'r').readlines()
    # strip off header, add to matrix 'spectra'
    spectra[i,:] = file_object[1:]                  
    
    # label spectrum class, based on header
    # actinolite: 0, alunite: 1
    material_name = file_object[0]
    
    # testcode
    # print(material_name)
    
    if material_name.find('Actinolite',)!= -1: #if material name contains actinolite
        y[i,0] = 0
    elif material_name.find('Alunite',)!= -1:
        y[i,0] = 1
    else:    
        y[i,0] = 2
               
    # turn deleted points into 0
    for j in range(spectrum_len):  
        if spectra[i,j] <0:
            spectra[i,j]=0                          
    i+=1

# test code
# print(y[:])
# print(spectra[0:2,0])

32


- Load data

In [104]:
# divide up data randomly

# 80% training data 20% test for this pilot
# eventually, 60% training set, 20% validation set, 20% test set
samples = list(range(0, num_samples))
train_set_size = 4*(num_samples//5)
test_set_size= num_samples-train_set_size
train_set_indices = random.sample(samples,train_set_size)
test_set_indices = np.setdiff1d(samples,train_set_indices)     #fixed bug: take remaining samples after making train set

# test code
# print(train_set_indices)
# print(test_set_indices)

# make train and test sets

train_set = spectra[train_set_indices, :]
train_labels = y[train_set_indices, :]
test_set = spectra[test_set_indices, :]
test_labels = y[test_set_indices, :]




- make model


In [105]:
# make model

# build model

model = keras.Sequential([
    keras.layers.Dense(10, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.relu),
    keras.layers.Dense(3, activation=tf.nn.softmax)
])

# compile
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train
model.fit(train_set, train_labels, epochs=15)

Epoch 1/15
24/24 [==============================] - 1s 43ms/step - loss: 1.0932 - acc: 0.4583
Epoch 2/15
24/24 [==============================] - 0s 208us/step - loss: 0.9341 - acc: 0.5000
Epoch 3/15
24/24 [==============================] - 0s 208us/step - loss: 0.9112 - acc: 0.5000
Epoch 4/15
24/24 [==============================] - 0s 208us/step - loss: 0.9345 - acc: 0.5000
Epoch 5/15
24/24 [==============================] - 0s 208us/step - loss: 0.9484 - acc: 0.5000
Epoch 6/15
24/24 [==============================] - 0s 209us/step - loss: 0.9475 - acc: 0.5000
Epoch 7/15
24/24 [==============================] - 0s 251us/step - loss: 0.9344 - acc: 0.5000
Epoch 8/15
24/24 [==============================] - 0s 208us/step - loss: 0.9155 - acc: 0.5000
Epoch 9/15
24/24 [==============================] - 0s 292us/step - loss: 0.8972 - acc: 0.5000
Epoch 10/15
24/24 [==============================] - 0s 167us/step - loss: 0.8859 - acc: 0.5000
Epoch 11/15
24/24 [==============================]

- run test

In [106]:
test_loss, test_acc = model.evaluate(test_set, test_labels)

print('Test accuracy:', test_acc)

8/8 [==============================] - 0s 51ms/step
Test accuracy: 0.375


- plot spectra